# **Descrição**

*   **Repositório contendo a abordagem completa:** https://github.com/imor-de/microsoft_malware_prediction_kaggle_2nd


*   **Abordagem de limpeza de dados utilizada:** https://github.com/imor-de/microsoft_malware_prediction_kaggle_2nd/blob/master/code/1_Data_Cleaning_train_set.ipynb

***
Nessa abordagem, MICHAELS; IMORDE realizam o processo de limpeza dos dados pré-processando o conjunto bruto e transformando-o para utilização de suas características. Com isso características com muitos valores NaN (Not a Number) são removidas, características com dimensões altamente desbalanceadas também são removidas e as características restantes são divididas em binárias, numéricas e categóricas.

Antes da limpeza do conjunto de dados, MICHAELS; IMORDE estudam os recursos e as características subjacentes de maneira intuitiva com processo de análise exploratória de dados. Investigando a distribuição da característica alvo; frequência na ocorrência de características binárias, numéricas e categóricas; dimensões de cada característica; frequência de dimensões presentes no conjunto de dados e frequência de valores NaN.

Com isso, características numéricas e binárias foram analisadas com base em valor máximo, valor mínimo, valor médio, desvio padrão de valores ocorridos e frequência de valores "0". Todas as informações interativas obtidas foram armazenadas (como descrições de características, comentários, tipos de características e informações sobre a limpeza dos dados) em um arquivo .xlsx separado durante o processo de análise exploratória de dados.
***

# **Bibliotecas/Funções**

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
import warnings, sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
# Especificação dos tipos de dados para menor uso de memória
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
def convert_types(df):
    # Conversão dos tipos de dados para reduzir memória utilizada
    for c in df:
        col_type = str(df[c].dtypes)
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        
        # Conversão de "object" para "category"
        if col_type == 'object':
            df[c] = df[c].astype('category')
        
        # Conversão de numéricos
        elif col_type in numerics:
            c_min = df[c].min()
            c_max = df[c].max()
            if col_type[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[c] = df[c].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[c] = df[c].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[c] = df[c].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[c] = df[c].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[c] = df[c].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[c] = df[c].astype(np.float32)
                else:
                    df[c] = df[c].astype(np.float64)  
        
    return df

# **Importação de Dados**

Para manter somente características importantes no conjunto de dados, os autores realizaram a exclusão de características com muitos valores NaN e também a exclusão de características com dimensões altamente desbalanceadas (como a característica “AutoSampleOptIn” onde o valor “0” é quase 100% presente).

In [ ]:
# Carregamento de informações sobre o conjunto de dados de um arquivo ".xlsx"
excel_table = pd.read_excel('MMP_Data_Description.xlsx')
excel_table = excel_table[excel_table['Relevant']==1]
relevant_features = [el.replace('\xa0','') for el in excel_table['Feature']]

In [ ]:
# Definição da coluna alvo
target = 'HasDetections'

# Definição da coluna identificadora
data_id = 'MachineIdentifier'

In [ ]:
# Atribuição da coluna alvo para as colunas relevantes presentes no arquivo xlsx
relevant_features.append(target)

In [ ]:
# Carregamento dos dados utilizando a biblioteca Dask para melhor performance
file = 'Microsoft_Malware_Prediction.csv'
ddf = dd.read_csv(file, dtype = dtypes)
df = ddf.compute()

In [ ]:
# Forma do conjunto de dados (linhas, colunas)
df.shape

(8921483, 83)

In [ ]:
# Remodelagem do conjunto de dados contendo somente as colunas relevantes presentes no arquivo .xlsx
df = df[relevant_features]

In [ ]:
# Forma do conjunto de dados remodelado (linhas, colunas)
df.shape

(8921483, 70)

In [ ]:
# Conversão dos tipos de dados para o conjunto remodelado
df = convert_types(df)

# **Limpeza de Dados**

In [ ]:
# Divisão das características em categóricas, binárias e numéricas (com base no arquivo .xlsx)
numerical_features = excel_table[excel_table['FeatureType']=='Numeric']['Feature'].reset_index(drop=True)
categorical_features = excel_table[excel_table['FeatureType']=='Category']['Feature'].reset_index(drop=True)
binary_features = excel_table[excel_table['FeatureType']=='Boolean']['Feature'].reset_index(drop=True)

In [ ]:
# Exibição das características binárias
binary_features

0                  Census_HasOpticalDiskDrive
1     Census_IsAlwaysOnAlwaysConnectedCapable
2                         Census_IsPenCapable
3            Census_IsPortableOperatingSystem
4                  Census_IsSecureBootEnabled
5                       Census_IsTouchEnabled
6                      Census_IsVirtualDevice
7                                    Firewall
8                                      HasTpm
9                                 IsProtected
10                           IsSxsPassiveMode
11                                      SMode
12                               Wdft_IsGamer
Name: Feature, dtype: object

In [ ]:
# Criação de listas para codificação e limpeza de dados
list_frequency_encoding = []
list_nan_minus1 = []
list_nan_0 = []
list_nan_1 = []

## **Numéricas**

In [ ]:
# Substituição de valores NaN por "-1" e armazenamento dessas características em "list_nan_minus1"
for feature in numerical_features:
    tmp1 = len(df)
    df[feature] = df[feature].fillna(-1)
    list_nan_minus1.append(feature)

## **Binárias**

In [ ]:
# Substituição de todos os valores NaN com o valor mais frequente de cada característica, armazenando as características em respectivas listas
for feature in binary_features:
    df[feature] = df[feature].fillna(df[feature].mode()[0])
    if df[feature].mode()[0] == 0:
        list_nan_0.append(feature)
    else:
        list_nan_1.append(feature)

## **Categóricas**

In [ ]:
# Renomeação de valores NaN como "-1" em todas as características que não são categóricas
correct_feature_by_hand = [] 
for feature in categorical_features:
    t = pd.api.types.is_categorical_dtype(df[feature])
    if not t:
        # Renomeação de valores NaN para "-1"
        df[feature] = df[feature].fillna(-1)
        list_nan_minus1.append(feature)
    else:
        # Adição à lista, para análise
        correct_feature_by_hand.append(feature)

## **Tratamento manual de características**

Para melhor aproveitamento e análise, MICHAELS; IMORDE modifica manualmente o conjunto de dados. Modificando valores nominais para caixa baixa; mesclando valores com diferentes ortografias; definindo valores NaN e valores com raras ocorrências para o termo "unknown"; rotulando todos os valores para valores numéricos e adicionando categorias para cada lista de codificação correspondente.





In [ ]:
correct_feature_by_hand

['AppVersion',
 'AvSigVersion',
 'Census_ActivationChannel',
 'Census_ChassisTypeName',
 'Census_DeviceFamily',
 'Census_FlightRing',
 'Census_GenuineStateName',
 'Census_MDC2FormFactor',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSInstallTypeName',
 'Census_OSSkuName',
 'Census_OSVersion',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_PowerPlatformRoleName',
 'Census_PrimaryDiskTypeName',
 'EngineVersion',
 'MachineIdentifier',
 'OsBuildLab',
 'OsPlatformSubRelease',
 'OsVer',
 'Platform',
 'Processor',
 'SkuEdition',
 'SmartScreen']

In [ ]:
for feature in correct_feature_by_hand:
    df[feature] = df[feature].str.lower()

In [ ]:
list_frequency_encoding.append('AppVersion')

In [ ]:
df['AvSigVersion'].replace(['1.2&#x17;3.1144.0'], '1.2173.1144.0', inplace=True)
list_frequency_encoding.append('AvSigVersion')

In [ ]:
def rename_Census_ActivationChannel(x):
    x = x.lower()
    if 'oem' in x:
        return 'oem'
    elif 'volume' in x:
        return 'volume'
    elif 'retail' in x:
        return 'retail'
    else:
        return x

In [ ]:
df['Census_ActivationChannel'] = df['Census_ActivationChannel'].astype(str)
df['Census_ActivationChannel'] = df['Census_ActivationChannel'].apply(rename_Census_ActivationChannel)
df['Census_ActivationChannel'] = df['Census_ActivationChannel'].astype('category')

In [ ]:
def rename_Census_ChassisTypeName(x):
    x = x.lower()
    if 'laptop' in x:
        return 'Notebook'
    elif 'other' in x:
        return 'unknown'                  
    else:
        return x

In [ ]:
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].fillna('unknown')
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].astype(str)
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].apply(rename_Census_ChassisTypeName)
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].astype('category')
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].cat.remove_unused_categories()

In [ ]:
df['Census_DeviceFamily'].replace(['windows'], 'windows.desktop', inplace=True)
df['Census_DeviceFamily'] = df['Census_DeviceFamily'].astype('category')
df['Census_DeviceFamily'] = df['Census_DeviceFamily'].cat.remove_unused_categories()

In [ ]:
df['Census_FlightRing'].replace(['disabled'], 'not_set', inplace=True)
df['Census_FlightRing'].replace(['osg', 'canary', 'invalid'], 'unknown', inplace=True)

df['Census_FlightRing'] = df['Census_FlightRing'].astype('category')

df['Census_FlightRing'] = df['Census_FlightRing'].fillna('unknown')
df['Census_FlightRing'] = df['Census_FlightRing'].cat.remove_unused_categories()

In [ ]:
df['Census_GenuineStateName'].replace(['tampered'], 'unknown', inplace=True)
df['Census_GenuineStateName'] = df['Census_GenuineStateName'].astype('category')
df['Census_GenuineStateName'] = df['Census_GenuineStateName'].fillna('unknown')
df['Census_GenuineStateName'] = df['Census_GenuineStateName'].cat.remove_unused_categories()

In [ ]:
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor']
correct_feature_by_hand.append('Census_MDC2FormFactor_new')

In [ ]:
def rename_Census_MDC2FormFactor_new(x):
    x = x.lower()
    if 'server' in x:
        return 'server'
    elif 'tablet' in x:
        return 'tablet'                  
    else:
        return x

In [ ]:
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].astype(str)
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].apply(rename_Census_MDC2FormFactor_new)
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].astype('category')
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].cat.remove_unused_categories()

In [ ]:
def rename_Census_OSEdition(x):
    x = x.lower()
    if 'core' in x:
        return 'core'
    elif 'pro' in x:
        return 'pro'
    elif 'enterprise' in x:
        return 'enterprise'
    elif 'server' in x:
        return 'server'
    elif 'home' in x:
        return 'home'
    elif 'education' in x:
        return 'education'
    elif 'cloud' in x:
        return 'cloud'
    else:
        return x

In [ ]:
df['Census_OSEdition'] = df['Census_OSEdition'].astype('category')
df['Census_OSEdition'] = df['Census_OSEdition'].cat.add_categories(['unknown'])
df['Census_OSEdition'] = df['Census_OSEdition'].fillna('unknown')
df['Census_OSEdition'] = df['Census_OSEdition'].astype(str)
df['Census_OSEdition'] = df['Census_OSEdition'].apply(rename_Census_OSEdition)
df['Census_OSEdition'] = df['Census_OSEdition'].astype('category')
df['Census_OSEdition'] = df['Census_OSEdition'].cat.remove_unused_categories()

In [ ]:
def rename_Census_OSSkuName(x):
    x = x.lower()
    if 'core' in x:
        return 'core'
    elif 'pro' in x:
        return 'pro'
    elif 'enterprise' in x:
        return 'enterprise'
    elif 'server' in x:
        return 'server'
    elif 'home' in x:
        return 'home'
    elif 'education' in x:
        return 'education'
    elif 'cloud' in x:
        return 'cloud'
    else:
        return x

In [ ]:
df['Census_OSSkuName'] = df['Census_OSSkuName'].astype(str)
df['Census_OSSkuName'] = df['Census_OSSkuName'].apply(rename_Census_OSSkuName)
df['Census_OSSkuName'] = df['Census_OSSkuName'].astype('category')
df['Census_OSSkuName'] = df['Census_OSSkuName'].cat.remove_unused_categories()

In [ ]:
list_frequency_encoding.append('Census_OSVersion')

In [ ]:
df['Census_PowerPlatformRoleName'].replace(['unspecified'], 'unknown', inplace=True)
df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].astype('category')
df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].fillna('unknown')
df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].cat.remove_unused_categories()

In [ ]:
df['Census_PrimaryDiskTypeName'].replace(['unspecified'], 'unknown', inplace=True)
df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].astype('category')
df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].fillna('unknown')
df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].cat.remove_unused_categories()

In [ ]:
list_frequency_encoding.append('EngineVersion')

In [ ]:
df['OsBuildLab'] = df['OsBuildLab'].astype('category')
df['OsBuildLab'] = df['OsBuildLab'].cat.add_categories(['unknown'])
df['OsBuildLab'] = df['OsBuildLab'].fillna('unknown')
list_frequency_encoding.append('OsBuildLab')

In [ ]:
def rename_SmartScreen(x):
    x = x.lower()
    if 'promt' in x:
        return 'prompt'
    elif 'requireadmin' in x:
        return 'requireadmin'
    elif 'existsnotset' in x:
        return 'existsnotset'
    elif 'off' in x:
        return 'off'
    elif 'warn' in x:
        return 'warn'
    elif 'prompt' in x:
        return 'prompt'
    elif 'block' in x:
        return 'block'
    elif 'on' in x:
        return 'on'   
    else:
        return 'unknown'

In [ ]:
df['SmartScreen'] = df['SmartScreen'].astype('category')
df['SmartScreen'] = df['SmartScreen'].cat.add_categories(['unknown'])
df['SmartScreen'] = df['SmartScreen'].fillna('unknown')
df['SmartScreen'] = df['SmartScreen'].astype(str)
df['SmartScreen'] = df['SmartScreen'].apply(rename_SmartScreen)
df['SmartScreen'] = df['SmartScreen'].astype('category')
df['SmartScreen'] = df['SmartScreen'].cat.remove_unused_categories()

In [ ]:
df.shape

(8921483, 71)

# **Exportação de Dados**

In [ ]:
# Exportação do conjunto de dados para o arquivo "train_cleaned.csv"
df.to_csv('MMP_Cleaned.csv', index = False)